In [ ]:
import pandas as pd

# 데이터 준비
california_df = pd.read_csv('California Housing Prices.csv')
california_df.head(), california_df.shape

In [ ]:
from sklearn.preprocessing import LabelEncoder

# object → int 
le_disaster = LabelEncoder()
california_df['ocean_proximity'] = le_disaster.fit_transform(california_df['ocean_proximity'])
california_df.info()

In [ ]:
california_df.describe()

In [ ]:
# 결측치 처리
california_df = california_df.dropna()
california_df.isnull().sum()


In [30]:
# 중복 데이터 확인 및 제거
# california_df.duplicated().sum()
# california_df.drop_duplicates(inplace=True)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
california_df.hist(bins=50, figsize=(20,15))
plt.show()

In [ ]:
# 이상치 확인
california_df.drop('median_house_value', axis=1).boxplot(figsize=(15,5))

In [33]:
# 이상치 제거
california_df = california_df[california_df['population'] <= 25000]

In [ ]:
# 상관관계
corr_mat = california_df.corr()
plt.figure(figsize = (10,8))
sns.heatmap(corr_mat, annot = True, cmap = 'coolwarm')
plt.show()

In [ ]:
# 왜도 확인
# california_df.skew()

import numpy as np
california_df['total_rooms'] = np.log1p(california_df['total_rooms'])
california_df['total_bedrooms'] = np.log1p(california_df['total_bedrooms'])
california_df['population'] = np.log1p(california_df['population'])
california_df['households'] = np.log1p(california_df['households'])
california_df.skew()

In [36]:
from sklearn.model_selection import train_test_split

X = california_df.drop(columns=['median_house_value'])
# X = california_df[['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income']]
y = california_df['median_house_value']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [37]:
# 데이터 스케일링
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [38]:
# 모델 훈련
# from sklearn.linear_model import LassoCV
# model = LassoCV(alphas=[0.01, 0.1, 1, 10, 100], cv=5)
# model.fit(X_train_scaled, y_train)
# model.alpha_

# from sklearn.linear_model import Lasso
# model = Lasso(alpha=10)

# from sklearn.linear_model import LinearRegression
# model = LinearRegression()

# from sklearn.linear_model import Ridge
# model = Ridge(alpha=1.0)

# from sklearn.linear_model import ElasticNet
# model = ElasticNet(alpha=0.1, l1_ratio=0.5)

# from sklearn.tree import DecisionTreeRegressor
# model = DecisionTreeRegressor()

# from sklearn.ensemble import RandomForestRegressor
# model = RandomForestRegressor(n_estimators=150)

from sklearn.ensemble import GradientBoostingRegressor
model = GradientBoostingRegressor()

# from sklearn.neural_network import MLPRegressor
# model = MLPRegressor(hidden_layer_sizes=(10, 10), max_iter=1000)

model.fit(X_train_scaled, y_train)

pred = model.predict(X_test_scaled)

In [39]:
# 앙상블
from xgboost import XGBRegressor

xgb_model = XGBRegressor(n_estimators=100, random_state=42)
xgb_model.fit(X_train_scaled, y_train)
xgb_pred = xgb_model.predict(X_test_scaled)

y_pred = (pred + xgb_pred) / 2

In [ ]:
# 훈련 정확도
train_acc = model.score(X_train_scaled, y_train)
train_acc

In [ ]:
# 테스트 정확도
test_acc = model.score(X_test_scaled, y_test)
test_acc

In [ ]:
# MSE
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(y_test, y_pred)
mse

In [ ]:
# r2_score
from sklearn.metrics import r2_score

r2 = r2_score(y_test, y_pred)
r2